In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

features = "features_new2.csv"
features = pd.read_csv(features, sep=',', header=0, index_col=None)
Y = features[['Output']]
# feat_not_theta = [i for i in features.columns if "Theta" not in i]
# X = features.drop(feat_not_theta, axis=1)
# not_time_feat = [i for i in features.columns if ("ERN" not in i) & ("Pe" not in i) & ("amp" not in i)]
# X = features.drop(not_time_feat, axis=1)

#not_freq_feat = [i for i in features.columns if ("Delta" not in i) & ("Theta" not in i) & ("Alpha" not in i) &
 #                ("Freq" not in i)]
#X = features.drop(not_freq_feat, axis=1)
# X = features.drop('Participant', axis=1)
X = features.drop('Output', axis=1)
print(len(X.columns)) 


1559


In [4]:
from mrmr import mrmr_classif
selected_features = mrmr_classif(X=X, y=Y, K=40)

100%|██████████| 40/40 [00:14<00:00,  2.69it/s]


In [5]:
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X[i]], axis=1)

In [ ]:
# Remove features with coefficient of variation < 0.2
variance = X.std()/X.mean()
low_variance = [i for i in variance.index if variance[i] < 0.2]
X = X.drop(low_variance, axis=1)
print(len(X.columns))

y0 = Y[Y['Output'] == 0]
feat_corr = np.zeros(len(X.columns))
feat_ttest = np.zeros(len(X.columns))

Y = np.ravel(Y)

i = 0
splits = 100
sss = StratifiedShuffleSplit(n_splits=splits, test_size=0.3, random_state=0)
for train_index, test_index in sss.split(X, Y):

    print(i)

    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    Y_train, Y_test = Y[train_index], Y[test_index]

    ind_y0 = [j for j in range(0, len(X_train)) if X_train.index[j] in y0.index]
    ind_y1 = [j for j in range(0, len(X_train)) if X_train.index[j] not in y0.index]
    X_ind_y0 = X_train.iloc[ind_y0, :]
    X_ind_y1 = X_train.iloc[ind_y1, :]

    corr = []

    # Remove correlated features
    for col1 in range(0, len(X.columns)-1):
        for col2 in range(col1+1, len(X.columns)):
            if abs(np.corrcoef(X_train[X.columns[col1]], X_train[X.columns[col2]])[0, 1]) > 0.9:
                if stats.ttest_ind(X_ind_y0[X.columns[col1]], X_ind_y1[X.columns[col1]]).pvalue < \
                        stats.ttest_ind(X_ind_y0[X.columns[col2]], X_ind_y1[X.columns[col2]]).pvalue:
                    if col2 not in corr:
                        feat_corr[col2] += 1
                        corr.append(col2)
                    elif col1 not in corr:
                        feat_corr[col1] += 1
                        corr.append(col1)

    # Relevance: t-test between independent variables and output
    ttest = [stats.ttest_ind(X_ind_y0[col], X_ind_y1[col])[1] for col in X.columns]
    ttest_order = np.argsort(ttest)
    feat_ttest += np.argsort(ttest_order)  # min(p-value) -> +0, max(p-value) -> +len(X.columns)-1

    i += 1

print(len(list(X.columns[feat_corr > splits/2])))
X = X.drop(list(X.columns[feat_corr > splits/2]), axis=1)  # Remove correlated features
feat_ttest = feat_ttest[feat_corr <= splits/2]

X = X.drop(X.columns[np.argsort(np.argsort(feat_ttest)) >= 40], axis=1)
feat_ttest = feat_ttest[np.argsort(np.argsort(feat_ttest)) < 40]
Xcol_sort = [x for _, x in sorted(zip(feat_ttest, X.columns))]
print(Xcol_sort)

In [6]:
#  ================================================== Classification ===================================================

f1score = []
precision = []
recall = []
specificity = []
npv = []
bal_acc = []

splits = 100
sss = StratifiedShuffleSplit(n_splits=splits, test_size=0.3, random_state=0)
svm = SVC(class_weight='balanced')
scaler = preprocessing.StandardScaler()
X_col = combined_df.columns
Y=np.array(Y)

for train_index, test_index in sss.split(combined_df, Y):
    X_train, X_test = combined_df.iloc[train_index, :], combined_df.iloc[test_index, :]
    Y_train, Y_test = Y[train_index], Y[test_index]
    scal = scaler.fit(X_train)
    X_train = scal.transform(X_train)  # Variables standardization
    X_test = scal.transform(X_test)  # Variables standardization
    X_train = pd.DataFrame(X_train, columns=X_col)
    X_test = pd.DataFrame(X_test, columns=X_col)
    clf = svm.fit(X_train, Y_train)
    Y_predicted = clf.predict(X_test)
    f1score.append(f1_score(Y_test, Y_predicted))
    precision.append(precision_score(Y_test, Y_predicted))  # Precision = Positive predictive value
    npv.append(precision_score(Y_test, Y_predicted, pos_label=0))  # Negative predictive value
    recall.append(recall_score(Y_test, Y_predicted))  # Recall = Sensitivity
    specificity.append(recall_score(Y_test, Y_predicted, pos_label=0))
    bal_acc.append(balanced_accuracy_score(Y_test, Y_predicted))

# print(np.mean(f1score))
# print(np.mean(precision))
print(np.mean(recall))
print(np.std(recall))
print(np.mean(specificity))
print(np.std(specificity))
# print(np.mean(npv))
print(np.mean(bal_acc))
print(np.std(bal_acc))

0.6341176470588237
0.0549567303843021
0.8830283911671925
0.022085083158282438
0.7585730191130078
0.025534166112890288
